### Import Libraries

In [ ]:
%pip install torch transformers pandas scipy
%pip install --upgrade --force-reinstall pillow

In [7]:
import pandas as pd
from transformers import pipeline, Trainer, TrainingArguments, AutoModelForCausalLM, AutoTokenizer
from torch.utils.data import Dataset, DataLoader

### Get the Dataset

In [9]:
data = {
    "prompt": [
        "Q: Pick up the red ball.\nA:",
        "Q: Grab the blue cube.\nA:",
        "Q: Take the green book.\nA:",
        "Q: Fetch the yellow toy.\nA:",
        "Q: What is the capital of France?\nA:",
        "Q: Who wrote 'Pride and Prejudice'?\nA:",
        "Q: Pick up the purple pen.\nA:",
        "Q: Pick up the orange ball.\nA:",
        "Q: Collect the gold star and the pearl.\nA:"
    ],
    "completion": [
        " pick up: red",
        " pick up: blue",
        " pick up: green",
        " pick up: yellow",
        " unknown",
        " unknown",
        " pick up: purple",
        " pick up: orange",
        " unknown"
    ]
}

df = pd.DataFrame(data)
df.to_csv("dataset.csv", index=False)

### Train the Model

In [10]:
# Custom Dataset class
class CustomDataset(Dataset):
    def __init__(self, dataframe, tokenizer, max_length):
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        prompt = self.dataframe.iloc[idx]['prompt']
        completion = self.dataframe.iloc[idx]['completion']
        inputs = self.tokenizer(prompt, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt")
        labels = self.tokenizer(completion, padding="max_length", truncation=True, max_length=self.max_length, return_tensors="pt").input_ids
        inputs['labels'] = labels
        return {key: val.squeeze() for key, val in inputs.items()}

In [13]:
# Load the dataset
dataset = pd.read_csv("dataset.csv")

# Load the model and tokenizer
model_name = "gpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Tokenize the dataset
max_length = 50
train_dataset = CustomDataset(dataset, tokenizer, max_length)

# Fine-tune the model
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    num_train_epochs=10,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
)

trainer.train()

/home/parth/.local/lib/python3.8/site-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>={ACCELERATE_MIN_VERSION}'`

### Save the Model

In [ ]:
model.save_pretrained("./llm-model-0")
tokenizer.save_pretrained("./llm-model-0")

### Load the Model

In [ ]:
# Load the fine-tuned model and tokenizer
model_name = "./llm-model-0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
generator = pipeline('text-generation', model=model, tokenizer=tokenizer)

# Function to generate a response based on the user's prompt
def generate_action(user_prompt):
    prompt = f"Q: {user_prompt}\nA:"
    response = generator(prompt, max_new_tokens=50, num_return_sequences=1, truncation=True)
    return response[0]['generated_text'].strip()

# Example user prompt
user_prompt = "Pick up the silver key."

# Generate and print the response
action = generate_action(user_prompt)
print(action)